In [9]:
import requests
import pandas as pd
import re

url = "https://raw.githubusercontent.com/Lorenzo1208/Projet_TrouveTonJob/main/data.json"
response = requests.get(url)
data = response.json()
df = pd.DataFrame(data)

df['salaire'] = df['lieu'].apply(lambda x: x[1] if len(x) > 1 else '')
df['lieu'] = df['lieu'].apply(lambda x : x[0])

df = df.apply(lambda c : c.apply(lambda x : ','.join([s.strip("\n ").lower() for s in (x.split(',') if type(x) != list else x)])))

df.to_csv("data_test1.csv", index=False)
df.head(24)

,Intitulé du poste,Date de publication,lieu,competences,Type de poste,salaire
0,data analyst - publicité,postée il y a 21 jours,paris,"sql,t-sql,outils",",,sept lieues sas,,cabinet de recrutement,,,cd...",
1,data analyst - h/f,postée il y a 1 mois,"paris,la défense","support,outils,agile,design,big data",",,edf france,,client final,,,cdi - temps plein",
2,customer data analyst (h/f),postée il y a 27 jours,paris,"crm,data quality,sql,sas,python",",,hermès,,client final,,,cdi - temps plein",
3,master data analyst h/f,postée il y a 21 jours,moussy-le-vieux,"si,support,sap",",,tereos,,client final,,,cdi - temps plein",
4,portfolio data analyst h/f,postée il y a 3 jours,france,"outils,support,decisionnel,sql,test",",,safran-group,,client final,,,cdi - temps plein",
5,data analyst bo/qlick sense (h/f),postée il y a 15 jours,maisons alfort,"support,sql,data management,business objects,t...",",,fed it,,cabinet de recrutement,,,cdd/intérim...","45.000,00 € - 50.000,00 € / an"
6,product data analyst - bi - h/f,postée il y a 28 jours,paris,"cloud,paas,qlik,iaas,support",",,cegid,,client final,,,cdi - temps plein",
7,data analyst / chef de projet bi (h/f),postée il y a 1 mois,paris,"data management,business intelligence,bases de...",",,fed it,,cabinet de recrutement,,,cdi - temps...","60.000,00 € - 62.000,00 € / an"
8,business analyst senior data crm - paris h/f,postée il y a 12 heures,paris,"crm,agile,decisionnel,réseaux,réseau",",,credit agricole,,client final,,,cdi - temps ...",
9,data scientist / analyst and software developm...,postée il y a 13 jours,rueil-malmaison,"big data,scrum,intelligence artificielle,elect...",",,vinci,,client final,,,stage",
